In [ ]:
# affine only
aug = iaa.SomeOf((0, 2), [
        iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}),
        iaa.Affine(translate_px={"x": (-15, 15), "y": (-15, 15)}),
        iaa.Affine(rotate=(-45, 45)),
        iaa.Affine(shear=(-20, 20)),
    ])

In [ ]:
aug = iaa.SomeOf((0, 3), [
        iaa.Fliplr(0.5), # horizontal flip
        iaa.Flipud(0.5), # vertical flip
        iaa.CropAndPad( # randomly crop up to 10 percent
            percent=(0, 0.1), 
            pad_mode=["constant", "edge"], # use constant value or closest edge pixel
            pad_cval=(0, 256)
        ),
        iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}),
        iaa.Affine(translate_px={"x": (-15, 15), "y": (-15, 15)}),
        iaa.Affine(rotate=(-45, 45)),
        iaa.Affine(shear=(-20, 20)),
        iaa.Multiply((0.5, 1.5), per_channel=0.5), # lighten or darken
        iaa.GaussianBlur(sigma=(0.0, 2.0)),
    ])

In [ ]:
aug = iaa.SomeOf((0, 3), [
        iaa.Fliplr(0.5), # horizontal flip
        iaa.Flipud(0.5), # vertical flip
        iaa.CropAndPad( # randomly crop up to 10 percent
            percent=(0, 0.1), 
            pad_mode=["constant", "edge"], # use constant value or closest edge pixel
            pad_cval=(0, 256)
        ),
        iaa.OneOf([iaa.Affine(rotate=90),
            iaa.Affine(rotate=180),
            iaa.Affine(rotate=270)]),
        iaa.Multiply((0.5, 1.5), per_channel=0.5), # lighten or darken
        iaa.Noop(),  # does nothing
        iaa.OneOf([ iaa.AdditiveGaussianNoise(scale=0.05*255),
            iaa.GaussianBlur(sigma=(0.0, 2.0)),
            iaa.Add((-20, 20), per_channel=0.5)]),  # change up pixels
    ])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()
data_gen_args = dict(rotation_range=90.,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2,
                         fill_mode='constant',
                         horizontal_flip=True,
                         vertical_flip=True,
                        )

In [ ]:
def get_train_test_augmented(X_data=X_train, Y_data=Y_train, validation_split=0.25, batch_size=32, seed=seed):
    X_train, X_test, Y_train, Y_test = train_test_split(X_data,
                                                        Y_data,
                                                        train_size=1-validation_split,
                                                        test_size=validation_split,
                                                        random_state=seed)
    
    # Image data generator distortion options
    data_gen_args = dict(rotation_range=45.,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='reflect')  #use 'constant'??


    # Train data, provide the same seed and keyword arguments to the fit and flow methods
    X_datagen = ImageDataGenerator(**data_gen_args)
    Y_datagen = ImageDataGenerator(**data_gen_args)
    X_datagen.fit(X_train, augment=True, seed=seed)
    Y_datagen.fit(Y_train, augment=True, seed=seed)
    X_train_augmented = X_datagen.flow(X_train, batch_size=batch_size, shuffle=True, seed=seed)
    Y_train_augmented = Y_datagen.flow(Y_train, batch_size=batch_size, shuffle=True, seed=seed)
     
    
    # Test data, no data augmentation, but we create a generator anyway
    X_datagen_val = ImageDataGenerator()
    Y_datagen_val = ImageDataGenerator()
    X_datagen_val.fit(X_test, augment=True, seed=seed)
    Y_datagen_val.fit(Y_test, augment=True, seed=seed)
    X_test_augmented = X_datagen_val.flow(X_test, batch_size=batch_size, shuffle=True, seed=seed)
    Y_test_augmented = Y_datagen_val.flow(Y_test, batch_size=batch_size, shuffle=True, seed=seed)
    
    
    # combine generators into one which yields image and masks
    train_generator = zip(X_train_augmented, Y_train_augmented)
    test_generator = zip(X_test_augmented, Y_test_augmented)
    
    return train_generator, test_generator